# Question 5: Consume Data Stream

In [ ]:
%pip install pyspark

In [4]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [2]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [3]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/06 18:21:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/ws/qbl6v0q52dxbcxsny29849_c0000gp/T/temporary-1956d80c-ca62-4d05-bb84-3c60c20d9a1d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 18:21:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/06 18:21:50 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 6, 14, 15, 38, 254000), timestampType=0)
DataFrame[lpep_pickup_datetime: string, lpep_dropoff_datetime: string, PULocationID: int, DOLocationID: int, passenger_count: double, trip_distance: double, tip_amount: double]


In [4]:
query.stop()

# Question 6

In [5]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [6]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [15]:
green_list = []

def peek(mini_batch, batch_id):
    print(mini_batch)
    for row in mini_batch.collect():
        green_list.append(row.asDict())

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/06 18:24:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/ws/qbl6v0q52dxbcxsny29849_c0000gp/T/temporary-2a5d704b-f3f8-46ac-a61a-1c13780fc08c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 18:24:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/06 18:24:24 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [21]:
len(green_list)

2858316

In [22]:
df_green = spark.createDataFrame(green_list)

In [23]:
df_green.show(10)

24/04/06 18:28:12 WARN TaskSetManager: Stage 124 contains a task of very large size (27272 KiB). The maximum recommended task size is 1000 KiB.
24/04/06 18:28:17 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 124 (TID 124): Attempting to kill Python Worker


+------------+------------+---------------------+--------------------+---------------+----------+-------------+
|DOLocationID|PULocationID|lpep_dropoff_datetime|lpep_pickup_datetime|passenger_count|tip_amount|trip_distance|
+------------+------------+---------------------+--------------------+---------------+----------+-------------+
|         196|         112|  2019-10-01 00:39:58| 2019-10-01 00:26:02|            1.0|       0.0|         5.88|
|         263|          43|  2019-10-01 00:22:38| 2019-10-01 00:18:11|            1.0|       0.0|          0.8|
|         228|         255|  2019-10-01 00:24:47| 2019-10-01 00:09:31|            2.0|       0.0|          7.5|
|         181|         181|  2019-10-01 00:41:49| 2019-10-01 00:37:40|            1.0|       0.0|          0.9|
|         188|          97|  2019-10-01 00:17:56| 2019-10-01 00:08:13|            1.0|      2.26|         2.52|
|          49|          65|  2019-10-01 00:43:40| 2019-10-01 00:35:01|            1.0|      1.86|       

In [24]:
query.stop()

24/04/06 19:14:19 WARN KafkaOffsetReaderAdmin: Error in attempt 1 getting Kafka offsets: 
java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: listOffsets on broker 1
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:396)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2073)
	at org.apache.kafka.common.internals.KafkaFutureImpl.get(KafkaFutureImpl.java:165)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.listOffsets(KafkaOffsetReaderAdmin.scala:88)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.$anonfun$fetchLatestOffsets$1(KafkaOffsetReaderAdmin.scala:332)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.$anonfun$partitionsAssignedToAdmin$1(KafkaOffsetReaderAdmin.scala:501)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.withRetries(KafkaOffsetReaderAdmin.scala:518)
	at org.apache.spark.sql.kafka010.

# Question 7

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql import functions as F

In [9]:
streamingInputDF = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [14]:
streamingDF = streamingInputDF \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [15]:
# Add a column "timestamp" using the `current_timestamp` function
streamingDF = streamingDF.withColumn("timestamp", current_timestamp())

In [16]:
# Group by 5 minutes window and "DOLocationID"
popular_destinations = streamingDF.groupBy(
    F.window(col("timestamp"), "5 minutes"),  # 5 minutes window
    "DOLocationID"
).count().orderBy("count", ascending=False)  # Order by count

In [20]:
import time
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(60)
query.stop()

24/04/06 22:39:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/ws/qbl6v0q52dxbcxsny29849_c0000gp/T/temporary-e46d88ff-0661-454d-bae2-bd54ac15fd9f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/06 22:39:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/06 22:39:33 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|74          |53223|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|42          |47826|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|41          |42183|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|75          |38520|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|129         |35790|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|7           |34599|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|166         |32535|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|236         |23739|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|223         |22626|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|238         |21954|
|{2024-04-06 22:35:00, 2024-04-06 22:40:00}|82          |21876|
|{2024-